## Library

In [ ]:
from confusion import *

In [ ]:
from my_models import *

In [ ]:
from geneval import *

In [ ]:
def scan_pythias(xs, prompt, steps, model_params = '70m', device = 'cpu',
                      n_iter = 8, out_length = 10, temperature = 1.):
    '''
    generate text for a parametrized prompt.
    xs: parameters
    my_prompt: a parametrizeable prompt
    
    returns dict with key = parameter, value = list of generated texts
    '''
    ys = {j: [] for j in xs}
    for j in xs:
        done = False
        while not done:
            try:
                model, tokenizer = my_pythia(STEP = steps(j), model_params = model_params, device = device)
                done = True
            except: print('retry')
        generated_output = f_generate_batch(model, tokenizer, [prompt]*n_iter, out_length,
                                            temperature = temperature)
        print(j, generated_output[0])
        ys[j] = (generated_output)
        del model
    return ys

In [ ]:
def logprobs_pythias(prompt, ys, xs, dxs, model_params = '70m', device = 'cpu'):
    '''
    return logprobs for prompts scan
    '''
    res = {}
    indexes = [(j,jj) for j in xs for jj in dxs]
    oldjjj = np.nan; model = np.nan
    for index in sorted(indexes, key = lambda x: x[0] + x[1]):# iterate in minimal model reloading sequence
        j, jj = index
        print(j,j+jj)
        if j+jj != oldjjj:
            del model
            model, tokenizer = my_pythia(STEP = steps(j+jj),
                                         model_params = model_params, device = device) #only load if j+jj changed
        oldjjj = j+jj
        f_eval_temp = f_eval_prob_batch(model, tokenizer, [prompt]*len(ys[j]), ys[j])
        for jjj, y_temp in enumerate(f_eval_temp):
            res[j, j+jj, jjj] = y_temp
    return res

## Program

### pythia scan

In [ ]:
prompt = ' '
n_params = '70m'
xs = list(range(0, 143, 1))
dxs = list(range(-1,2,1))
n_iter = 8*128

def steps(x): return np.minimum(142000, np.maximum(0, 1000*x))

In [ ]:
ys = scan_pythias(xs, prompt, steps, model_params = n_params, device ='cpu',
              n_iter = n_iter, out_length = 10, temperature = 1.)


res = logprobs_pythias(prompt, ys, xs, dxs, model_params = n_params, device = 'cpu')

In [ ]:
# calculate dissimilarity g with L-parameter L
L=1
Y, dY = stat_indicator_scan_final(xs,ys,res,L, g = lambda x: 2*x-1)

In [ ]:
# plot
import pylab as plt
plt.plot(0.5*(np.array(xs[L:])+np.array(xs[:-L])), Y, 'o--')
plt.xlabel('epcoh [1000]')
plt.ylabel('dissimilarity')